In [1]:
import pandas as pd
import numpy as np
import os
from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [2]:
results_dir = 'C:\\Users\\RonyArmon\\Projects_Code\\Cluster_Activities\\results'
pos_df = pd.read_pickle(os.path.join(results_dir,'pos.pkl'))
tokens_similarity = pd.read_pickle(os.path.join(results_dir,'tokens_similarity.pkl'))
lemmas_synonyms = pd.read_excel(os.path.join(results_dir, 'lemmas_synonyms.xlsx'))

In [3]:
print(pos_df.head())

   token_id  name_id loc      token      lemma   upos xpos head deprel  \
0         1        1   1  construct  construct   VERB   VB    0   root   
0         2        1   2     trials      trial   NOUN  NNS    1    obj   
0         3        1   3          &          &  CCONJ   CC    4     cc   
0         4        1   4       test       test   NOUN   NN    2   conj   
0         5        1   5        for        for    ADP   IN    7   case   

  start_char end_char                  feats  
0          0        9  Mood=Imp|VerbForm=Fin  
0         10       16            Number=Plur  
0         17       18                      _  
0         19       23            Number=Sing  
0         24       27                      _  


In [4]:
# Sentences
df1 = pos_df[pos_df['name_id']==1]
df2 = pos_df[pos_df['name_id']==2]
tokens1, tokens2 = list(df1['token']), list(df2['token'])
if len(tokens1)<=len(tokens2):
    tokens_s, tokens_l = tokens1, tokens2
else:
    tokens_s, tokens_l = tokens2, tokens1
print('short name tokens:', tokens_s)
print('long name tokens:', tokens_l)

short name tokens: ['turbine', 'hall', '(', 'driven', 'piles', ')']
long name tokens: ['construct', 'trials', '&', 'test', 'for', 'bd', 'inspection', 'and', 'acknowledgement']


In [6]:
token_pairs = []
for token1 in tokens_s:
    checked, scores = [], {}
    for token2 in tokens_l:
        if token2 not in checked:
            score = tokens_similarity.at[token1, token2]
            checked.append(token2)
            print(token1, token2, score)
            if score>0.3: 
                scores[token2]=score
    scores_array = np.array(list(scores.values()))
    if np.sum(scores_array)>0:
        max_score = np.max(scores_array)
        print('max score:', max_score)
        match_score = {k:v for k,v in scores.items() if v == max_score}
        print ('match_score:', match_score)
        ms_keys, ms_vals = list(match_score.keys()), list(match_score.values())
        match_score = (token1, ms_keys[0], ms_vals[0])
        token_pairs.append(match_score)
    else:
        print('no match')

turbine construct 0.25
turbine trials 0.46
turbine & 0.0
turbine test 0.36
turbine for 0.2
turbine bd 0.22
turbine inspection 0.35
turbine and 0.2
turbine acknowledgement 0.09
max score: 0.46
match_score: {'trials': 0.46}
hall construct 0.0
hall trials 0.4
hall & 0.0
hall test 0.0
hall for 0.0
hall bd 0.0
hall inspection 0.0
hall and 0.29
hall acknowledgement 0.21
max score: 0.4
match_score: {'trials': 0.4}
( construct 0.0
( trials 0.0
( & 0.0
( test 0.0
( for 0.0
( bd 0.0
( inspection 0.0
( and 0.0
( acknowledgement 0.0
no match
driven construct 0.13
driven trials 0.33
driven & 0.0
driven test 0.2
driven for 0.22
driven bd 0.25
driven inspection 0.38
driven and 0.22
driven acknowledgement 0.29
max score: 0.38
match_score: {'inspection': 0.38}
piles construct 0.14
piles trials 0.55
piles & 0.0
piles test 0.44
piles for 0.0
piles bd 0.0
piles inspection 0.27
piles and 0.0
piles acknowledgement 0.2
max score: 0.55
match_score: {'trials': 0.55}
) construct 0.0
) trials 0.0
) & 0.0
) test 

In [7]:
token_pairs

[('turbine', 'trials', 0.46),
 ('hall', 'trials', 0.4),
 ('driven', 'inspection', 0.38),
 ('piles', 'trials', 0.55)]